# Manager -> **Query pre process**

In [1]:
import sys
sys.path.append("C:\\Users\\lauth\\OneDrive\\Desktop\\sql_assistant_v3")

from src.components.models.llms.llms import Langchain_OpenAI_LLM, HF_Llama38b_LLM
from src.components.models.embeddings.embeddings import HF_MultilingualE5_Embeddings, Openai_Embeddings
from src.components.memory.memory import Memory
from src.app.app import AppDataCollector

langchain_llm = Langchain_OpenAI_LLM()
langchain_llm.init_model()

hf_llm = HF_Llama38b_LLM()
hf_llm.init_model()

mle5_embeddings = HF_MultilingualE5_Embeddings()
mle5_embeddings.init_model()

openai_embeddings = Openai_Embeddings()
openai_embeddings.init_model()

memory = Memory()
collector = AppDataCollector()


In [2]:
memory.add_user_message("I want information from EMED-2012-LR-P12")

In [2]:
collector.user_request = (
    "The human is requesting information from the FQI-EMED-3135.01-067"
)
collector.modified_user_request = (
    "The human is requesting information from the FQI-EMED-3135.01-067 flow computer."
)
collector.flavored_request_for_semantic_search = (
    '"The human is requesting information from the firmware."'
)
collector.semantic_info = {
    "fcs_firmware": [
        (
            "Id",
            "INT",
            "Unique identifier for each flow computer firmware",
            1,
            "fcs_firmware",
        ),
        (
            "Id",
            "INT",
            "Unique identifier for each flow computer firmware",
            1,
            ("fcs_computadores", "fcs_firmware"),
        ),
        (
            "Firmware",
            "VARCHAR(100)",
            "Name or version of the firmware",
            2,
            "fcs_firmware",
        ),
        (
            "Estado",
            "VARCHAR(10)",
            "Status of the firmware, active ('ACT') or inactive ('INA')",
            3,
            "fcs_firmware",
        ),
        (
            "IdTipo_Computador",
            "INT",
            "Foreign key linking firmwares with flow computer types",
            999,
            ("fcs_firmware", "fcs_tipo_computador"),
        ),
    ],
    "fcs_computadores": [
        (
            "Id",
            "INT",
            "Unique identifier for each flow computer",
            1,
            "fcs_computadores",
        ),
        (
            "IP",
            "VARCHAR(100)",
            "Flow computer IP address of the flow computer",
            2,
            "fcs_computadores",
        ),
        (
            "Puerto",
            "INT",
            "Flow computer primary port used for communications",
            3,
            "fcs_computadores",
        ),
        ("Tag", "VARCHAR(100)", "Flow computer name", 4, "fcs_computadores"),
        (
            "Estado",
            "VARCHAR(100)",
            "Current status of the flow computer, active ('1') or inactive ('2')",
            5,
            "fcs_computadores",
        ),
        (
            "IdFirmware_fk",
            "INT",
            "Foreign key linking flow computers with respective firmware",
            999,
            ("fcs_computadores", "fcs_firmware"),
        ),
    ],
}

collector.sql_code="SELECT c.Tag, f.Firmware AS 'Revision/Version', tc.Nombre AS Marca FROM dbo_v2.fcs_computadores c JOIN dbo_v2.fcs_firmware f ON f.Id = c.IdFirmware_fk JOIN dbo_v2.fcs_tipo_computador tc ON tc.Id = f.IdTipo_Computador_fk WHERE c.Tag = 'FQI-EMED-3135.01-067'"

collector.assistant_sql_code_suggestion = 'Please replace the placeholder "<Flow computer Tag>" with the actual tag of the flow computer you are interested in.'

collector.assistant_sql_code_analysis='The query is almost complete and ready for execution, but it needs one more piece of information, "Flow computer Tag", to be specified.'

collector.ai_pre_response = "The FQI-EMED-3135.01-067 flow computer is a FB107 revision/version manufactured by FLOWBOSS."

collector.ai_post_response = "El computador de flujo FQI-EMED-3135.01-067 es una revisión/versión FB107 fabricada por FLOWBOSS."



In [3]:
from src.app.pipeline_processes.query_post_process.generation import translate_response
from src.app.pipeline_processes.sql_post_process.manager import complex_request_pre_query_generation, complex_request_sql_verification
from src.components.models.models_interfaces import Base_LLM
from src.app.pipeline_processes.sql_post_process.generation import generate_sql_classification, generate_sql_pre_query, generate_sql_summary_response
from src.app.pipeline_processes.sql_generation_process.manager import (
    complex_request_sql_generation,
)
from src.app.pipeline_processes.sql_pre_process.manager import (
    complex_request_process_modification,
    complex_request_process_semantics,
)

from src.app.pipeline_processes.query_pre_process.manager import (
    query_pre_process,
    simple_request_process,
)


from src.components.models.llms.llms import HF_Llama38b_LLM
from src.settings.settings import Settings

import pandas as pd


"""Para pre procesar, obtener el request y clasificacion del request se usa: """
# query_pre_process(langchain_llm, memory, collector)


"""Si el request es simple, se usa: """
# simple_request_process(langchain_llm, memory, collector)



"""Si el request es complejo se usa: """
# ? Para modificar el request:
# complex_request_process_modification(hf_llm, mle5_embeddings, collector)


# ? Para buscar tablas e informacion semantica de la base sql en la base vectorial
# complex_request_process_semantics(langchain_llm, openai_embeddings, collector)


# ? Para generar el sql en primera instancia
# complex_request_sql_generation(langchain_llm, collector)

# ? Para verificar la clasificacion del sql completo o incompleto
# complex_request_sql_verification(hf_llm, collector)

# ? Para generando un pre query si el sql es incompleto
# complex_request_pre_query_generation(langchain_llm, collector)


# ? Para generar la respuesta de resumen al dtaframe del sql
# primero se debe ejecutar el sql para obtener el dataframe df
# df = run_sql(collector.sql_code) o run_sql(collector.sql_pre_query)
# complex_request_sql_summary_response(langchain_llm, collector, df)

"""Post procesando la respuesta: """
# query_post_process(langchain_llm, collector, "Quiero saber informacion de FQI-EMED-3135.01-067")
